In [ ]:
"""
Purpose: Run the Soma Finding
Algorithm for all the cells
in our final match



"""


In [1]:
current_version = 30

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

INFO - 2021-03-04 19:50:39,994 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-04 19:50:39,997 - settings - Setting database.user to celiib
INFO - 2021-03-04 19:50:39,998 - settings - Setting database.password to newceliipass
INFO - 2021-03-04 19:50:40,001 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-04 19:50:40,002 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-04 19:50:40,017 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-04 19:50:40,429 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-04 19:50:40,433 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-04 19:50:40,434 - settings - Setting database.user to celiib
INFO - 2021-03-04 19:50:40,434 - settings - Setting database.password to newceliipass
INFO - 2021-03-04 19:50:40,435 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-04 19:50:40,436 - settings - Setting database.user to celiib
INFO - 2021-03-04 19:50:40,438 - settings - Setting database.password to newceliipass
INFO - 2021-03-04 19:50:40,440 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-04 19:50:40,886 - settings - Setting enable_python_native_blobs to True


In [3]:
test_mode = True

In [4]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-04 19:50:40,971 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-04 19:50:40,972 - settings - Setting database.user to celiib
INFO - 2021-03-04 19:50:40,973 - settings - Setting database.password to newceliipass
INFO - 2021-03-04 19:50:40,974 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-04 19:50:40,977 - settings - Setting database.user to celiib
INFO - 2021-03-04 19:50:40,978 - settings - Setting database.password to newceliipass
INFO - 2021-03-04 19:50:40,979 - settings - Setting enable_python_native_blobs to True


Sleeping 110 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-03-04 19:50:41,458 - settings - Setting enable_python_native_blobs to True


# Getting the list of neurons to decompose for the mutli soma testing

In [5]:
# import pandas as pd
# soma_soma_table = pd.read_csv("Minnie65 core proofreading - Soma-Soma.csv")
# no_header = soma_soma_table.iloc[1:]
# multi_soma_ids_str = no_header["Dendrites"].to_numpy()
# multi_soma_ids = multi_soma_ids_str[~np.isnan(multi_soma_ids_str.astype("float"))].astype("int")

# @schema
# class MultiSomaProofread(dj.Manual):
#     definition="""
#     segment_id : bigint unsigned  #segment id for those to be decimated
#     """
    
# dict_of_seg = [dict(segment_id=k) for k in multi_soma_ids]
# minnie.MultiSomaProofread.insert(dict_of_seg,skip_duplicates=True)
# MultiSomaProofread()

# Defining the Table

In [6]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [7]:
import meshlab
meshlab.set_meshlab_port(current_port=None)
temporary_folder = 'decimation_temp'
meshlab_scripts = {}

No port chosen so picked random port 1859


In [8]:
#so that it will have the adapter defined
from datajoint_utils import *

In [9]:
@schema
class NeuronGliaNuclei(dj.Manual):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ver : decimal(6,2) #the version number of the materializaiton
    ---
    n_glia_faces              : int unsigned                 # The number of faces that were saved off as belonging to glia
    glia_faces=NULL           : <faces>                      # faces indices that were saved off as belonging to glia (external storage)
    n_nuclei_faces            : int unsigned                 # The number of faces that were saved off as belonging to nuclie
    nuclei_faces=NULL         : <faces>                      # faces indices that were saved off as belonging to nuclei (external storage)
    """

In [10]:
# schema.external['faces'].delete(delete_external_files=True)
# schema.external['somas'].delete(delete_external_files=True)

In [11]:
# minnie.BaylorSegmentCentroid.delete()
# minnie.NeuronGliaNuclei().delete()

In [16]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
current_version = 30

key_source = (((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") )  & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj())

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),soma_index index given to this soma to account for multiple somas in one base semgnet,ver the version number of the materializaiton,centroid_x (EM voxels),centroid_y (EM voxels),centroid_z (EM voxels),n_vertices number of vertices,n_faces number of faces,mesh datajoint adapter to get the somas mesh objects,multiplicity the number of somas found for this base segment,sdf sdf width value for the soma,volume the volume in billions (10*9 nm^3) of the convex hull,max_side_ratio the maximum of the side length ratios used for check if soma,bbox_volume_ratio ratio of bbox (axis aligned) volume to mesh volume to use for check if soma,max_hole_length euclidean distance of the maximum hole size,run_time the amount of time to run (seconds)
864691134988608634,0,0.25,1,30.00,151978,185713,26304,21523,42748,=BLOB=,2,0.744,1369.9483234182394,1.138,5.259,164323.827,643.1645
864691134988608634,0,0.25,2,30.00,145965,170142,26401,18288,36394,=BLOB=,2,0.885,1090.709593370298,1.092,5.773,41529.935,643.1645
864691135065024068,0,0.25,1,30.00,256704,178828,18168,46789,91851,=BLOB=,1,0.746,4869.565858574333,1.178,5.073,7651221.993,481.3812
864691135155711844,0,0.25,1,30.00,213707,116033,21601,38102,75062,=BLOB=,1,0.87,3212.236160781479,1.228,7.616,970650.073,367.7925
864691135210232512,0,0.25,1,30.00,225921,206935,17407,24065,73651,=BLOB=,2,0.576,1875.1993309595885,1.445,3.048,260616.601,370.6695
864691135210232512,0,0.25,2,30.00,233198,221309,18533,7834,14450,=BLOB=,2,0.705,455.40817405,1.39,5.524,1892493.179,370.6695
864691135341057989,0,0.25,1,30.00,187888,154907,19639,14109,28039,=BLOB=,2,0.884,965.9089776211927,1.497,4.768,48356.196,296.7842
864691135341057989,0,0.25,2,30.00,201482,125642,20329,15466,30460,=BLOB=,2,0.884,981.7812867800417,1.313,4.181,69246.747,296.7842
864691135345291423,0,0.25,1,30.00,196623,127334,24613,17425,34530,=BLOB=,1,0.787,1031.2078735189218,1.25,6.117,63494.512,359.7465
864691135373423816,0,0.25,1,30.00,237920,168543,21108,23056,45474,=BLOB=,1,0.859,1579.2820321304062,1.147,3.991,149462.436,1295.5624


In [13]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000
current_version = 30


import trimesh_utils as tu
import soma_extraction_utils as sm
@schema
class BaylorSegmentCentroid(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    soma_index : tinyint unsigned #index given to this soma to account for multiple somas in one base semgnet
    ver : decimal(6,2) #the version number of the materializaiton
    ---
    centroid_x=NULL           : int unsigned                 # (EM voxels)
    centroid_y=NULL           : int unsigned                 # (EM voxels)
    centroid_z=NULL           : int unsigned                 # (EM voxels)
    n_vertices=NULL           : bigint                 #number of vertices
    n_faces=NULL            : bigint                  #number of faces
    mesh: <somas>  #datajoint adapter to get the somas mesh objects
    multiplicity=NULL         : tinyint unsigned             # the number of somas found for this base segment
    sdf=NULL                  : double                       # sdf width value for the soma
    volume=NULL               : double                       # the volume in billions (10*9 nm^3) of the convex hull
    max_side_ratio=NULL       : double                       # the maximum of the side length ratios used for check if soma
    bbox_volume_ratio=NULL    : double                       # ratio of bbox (axis aligned) volume to mesh volume to use for check if soma
    max_hole_length=NULL      : double                    #euclidean distance of the maximum hole size
    run_time=NULL : double                   # the amount of time to run (seconds)

    """

    key_source = (((minnie.Decimation & f"n_vertices > {verts_min}").proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}") & (dj.U("segment_id") )  & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj())
                                                                 
     

    def make(self,key):
        """
        Pseudocode: 
        1) Compute all of the
        2) Save the mesh as an h5 py file
        3) Store the saved path as the decomposition part of the dictionary and erase the vertices and faces
        4) Insert
        
        
        """
        
        #get the mesh data
        print(f"\n\n\n---- Working on Neuron {key['segment_id']} ----")
        print(key)
        new_mesh = (minnie.Decimation() & key).fetch1("mesh")
        current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces

        segment_id = key["segment_id"]

        (total_soma_list, 
         run_time, 
         total_soma_list_sdf,
         glia_pieces,
         nuclei_pieces) = sm.extract_soma_center(
                            segment_id,
                            current_mesh_verts,
                            current_mesh_faces,
            return_glia_nuclei_pieces=True,
        )
        
        # -------- 1/9 Addition: Going to save off the glia and nuclei pieces ----------- #
        """
        Psuedocode:
        For both glia and nuclie pieces
        1) If the length of array is greater than 0 --> combine the mesh and map the indices to original mesh
        2) If not then just put None     
        """
        orig_mesh = trimesh.Trimesh(vertices=current_mesh_verts,
                                   faces=current_mesh_faces)
        
        if len(glia_pieces)>0:
            glia_faces = tu.original_mesh_faces_map(orig_mesh,tu.combine_meshes(glia_pieces))
            n_glia_faces = len(glia_faces)
        else:
            glia_faces = None
            n_glia_faces = 0
            
        if len(nuclei_pieces)>0:
            nuclei_faces = tu.original_mesh_faces_map(orig_mesh,tu.combine_meshes(nuclei_pieces))
            n_nuclei_faces = len(nuclei_faces)
        else:
            nuclei_faces = None
            n_nuclei_faces = 0
            
        # --------- saving the nuclei and glia saves
        glia_path,nuclei_path = du.save_glia_nuclei_files(glia_faces=glia_faces,
                                 nuclei_faces=nuclei_faces,
                                 segment_id=segment_id)
        
        print(f" glia_path = {glia_path} \n nuclei_path = {nuclei_path}")
            
        glia_nuclei_key = dict(key,
                               ver=current_version,
                               n_glia_faces=n_glia_faces,
                               #glia_faces = glia_faces,
                               glia_faces = glia_path,
                               n_nuclei_faces = n_nuclei_faces,
                               #nuclei_faces = nuclei_faces
                               nuclei_faces = nuclei_path,
                              )
        
        NeuronGliaNuclei.insert1(glia_nuclei_key,replace=True)
        print(f"Finished saving off glia and nuclei information : {glia_nuclei_key}")
        
        # ---------------- End of 1/9 Addition --------------------------------- #
        
        
        
        print(f"Run time was {run_time} \n    total_soma_list = {total_soma_list}"
             f"\n    with sdf values = {total_soma_list_sdf}")
        
        #check if soma list is empty and did not find soma
        if len(total_soma_list) <= 0:
            print("There were no somas found for this mesh so just writing empty data")
            

            returned_file_path = tu.write_h5_file(
                                                vertices=np.array([]),
                                                  faces=np.array([]),
                                                  segment_id=segment_id,
                                                  filename = f'{segment_id}_0.h5',
                                                    filepath=str(du.get_somas_path())
                                                 )

            
            
            insert_dict = dict(key,
                              soma_index=0,
                               ver=current_version,
                              centroid_x=None,
                               centroid_y=None,
                               centroid_z=None,
                               #distance_from_prediction=None,
                               #prediction_matching_index = None,
                               n_vertices=0,
                               n_faces=0,
                               mesh=returned_file_path,
                               multiplicity=0,
                               sdf = None,
                               volume = None,
                               max_side_ratio = None,
                               bbox_volume_ratio = None,
                               max_hole_length=None,
                               run_time=run_time
                              )
            
            #raise Exception("to prevent writing because none were found")
            self.insert1(insert_dict,skip_duplicates=True)
            return
        
        #if there is one or more soma found, get the volume and side length checks
        max_side_ratio =  [np.max(sm.side_length_ratios(m)) for m in total_soma_list]
        bbox_volume_ratio =  [sm.soma_volume_ratio(m) for m in total_soma_list]
        dicts_to_insert = []


        for i,(current_soma,soma_sdf,sz_ratio,vol_ratio) in enumerate(zip(total_soma_list,total_soma_list_sdf,max_side_ratio,bbox_volume_ratio)):
            print("Trying to write off file")
            """ Currently don't need to export the meshes
            current_soma.export(f"{key['segment_id']}/{key['segment_id']}_soma_{i}.off")
            """
            auto_prediction_center = np.mean(current_soma.vertices,axis=0) / np.array([4,4,40])
            auto_prediction_center = auto_prediction_center.astype("int")
            print(f"Predicted Coordinates are {auto_prediction_center}")
            max_hole_length = tu.largest_hole_length(current_soma)
            
            returned_file_path = tu.write_h5_file(
                                            vertices=current_soma.vertices,
                                              faces=current_soma.faces,
                                              segment_id=segment_id,
                                              filename = f'{segment_id}_{i}.h5',
                                                filepath=str(du.get_somas_path())
                                             )



            insert_dict = dict(key,
                              soma_index=i+1,
                               ver=current_version,
                              centroid_x=auto_prediction_center[0],
                               centroid_y=auto_prediction_center[1],
                               centroid_z=auto_prediction_center[2],
                               n_vertices = len(current_soma.vertices),
                               n_faces = len(current_soma.faces),
                               mesh=returned_file_path,
                               multiplicity=len(total_soma_list),
                               sdf = np.round(soma_sdf,3),
                               volume = current_soma.convex_hull.volume/1000000000,
                               max_side_ratio = np.round(sz_ratio,3),
                               bbox_volume_ratio = np.round(vol_ratio,3),
                               max_hole_length = np.round(max_hole_length,3),
                               run_time=np.round(run_time,4)
                              )



            dicts_to_insert.append(insert_dict)
        self.insert(dicts_to_insert,skip_duplicates=True)

# Running the Populate

In [14]:
curr_table = (minnie.schema.jobs & "table_name='__baylor_segment_centroid'")
(curr_table)#.delete()#.delete()# & "status='error'"#.delete()
#curr_table.delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__baylor_segment_centroid,650062339c7ed1c5ae3c0180706af144,reserved,=BLOB=,,=BLOB=,celiib@10.28.0.83,at-compute004,23385,202338,2021-03-04 13:50:33


In [15]:
import time
import random

sm = reload(sm)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    BaylorSegmentCentroid.populate(reserve_jobs=True, suppress_errors=True)
else:
    BaylorSegmentCentroid.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for BaylorSegmentCentroid populate = {time.time() - start_time}")

INFO - 2021-03-04 19:50:56,599 - autopopulate - Found 8 keys to populate
INFO - 2021-03-04 19:50:56,610 - connection - Transaction started
INFO - 2021-03-04 19:50:56,612 - autopopulate - Populating: {'segment_id': 864691135564804951, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}


Populate Started



---- Working on Neuron 864691135564804951 ----
{'segment_id': 864691135564804951, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25')}
Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 5000.0 
large_mesh_threshold_inner = 3250.0 
soma_size_threshold = 562.5 
soma_size_threshold_max = 75000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
max_mesh_sized_filtered_away = 22500.0
xvfb-run -n 1859 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_31365.off -o /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_31365_remove_interior.off -s /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/remove_interior_711893.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_31365.off
r

/meshAfterParty/meshAfterParty/trimesh_utils.py:685: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ordered_comp_indices = np.array([k.astype("int") for k in ordered_components])


There were 24 total interior meshes
Pieces satisfying glia requirements (volume) (x >= 2500000000000): 1
Pieces satisfying nuclie requirements: n_faces (700 <= x) and volume (x < 2500000000000) : 23
using precomputed glia pieces

 ---- Working on inside piece 0 ------
For glia mesh 0 there were 581747 faces within 3000 distane
New mesh size is <trimesh.Trimesh(vertices.shape=(290961, 3), faces.shape=(581747, 3))>
Found 2179 and going to remove 2086 that were inside bounding box
After removal of floating pieces the mesh is <trimesh.Trimesh(vertices.shape=(122100, 3), faces.shape=(242900, 3))>
inside remove_mesh_interior and using precomputed inside_pieces
Removing the following inside neurons: [<trimesh.Trimesh(vertices.shape=(3303, 3), faces.shape=(5949, 3))>, <trimesh.Trimesh(vertices.shape=(2250, 3), faces.shape=(4366, 3))>, <trimesh.Trimesh(vertices.shape=(1560, 3), faces.shape=(2442, 3))>, <trimesh.Trimesh(vertices.shape=(1368, 3), faces.shape=(2246, 3))>, <trimesh.Trimesh(vertices

  0%|          | 0/6 [00:00<?, ?it/s]

...

Largest hole before segmentation = 69706.15028774554, after = 331463.16951856104,

ratio = 4.755149554957317, difference = 261757.01923081552
Skipping the segmentatio filter at end
removing mesh interior before segmentation
xvfb-run -n 1859 -s "-screen 0 800x600x24" meshlabserver $@  -i /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_1796.off -o /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_1796_fill_holes.off -s /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/fill_holes_628241.mls
removed temporary input file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_1796.off
removed temporary output file: /neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/temp/neuron_1796_fill_holes.off
/neuron_mesh_tools/Auto_Proofreading/Preprocessing_Pipeline/Allen_Splitting_Pipeline/te

  0%|          | 0/10 [00:00<?, ?it/s]

Largest hole before segmentation = 5925960.875684581, after = 5782844.233407164,

ratio = 0.975849209051539, difference = -143116.64227741677
Saved object at /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137196942401_glia.pbz2
File size is 4.6e-05 MB
Saved object at /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137196942401_nuclei.pbz2
File size is 0.147235 MB
 glia_path = /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137196942401_glia.pbz2 
 nuclei_path = /mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137196942401_nuclei.pbz2
Finished saving off glia and nuclei information : {'segment_id': 864691137196942401, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': 30, 'n_glia_faces': 0, 'glia_faces': PosixPath('/mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137196942401_glia.pbz2'), 'n_nuclei_faces': 94826, 'nuclei_faces': PosixPath('/mnt/dj-stor01/platinum/minnie65/02/glia_nuclei_faces/864691137196942401

/meshAfterParty/meshAfterParty/networkx_utils.py:593: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  cycles_list_array = np.array(cycles_list)
INFO - 2021-03-04 20:33:31,758 - connection - Transaction committed and closed.


Populate Done
Total time for BaylorSegmentCentroid populate = 2555.188392162323
